In [ ]:
from base58 import b58decode, b58encode

from solanaweb3.message import *
from solanaweb3.blockhash import Blockhash
from solanaweb3.publickey import PublicKey
from solanaweb3.transaction import Transaction, SIG_LENGTH

def print_message(msg):
    print("Header: ", msg.header)
    print("Recent Blockhash: ", b58decode(msg.recent_blockhash).hex())
    any((print(f"key{i + 1}: {key.to_buffer().hex()}") for i, key in enumerate(msg.account_keys)))
    print("Instructions: ", msg.instructions)

In [ ]:
message_header = MessageHeader(0, 3, 2)
accounts_keys = [str(PublicKey(i)) for i in range(1, 6)]
recent_blockhash = Blockhash(str(PublicKey(1)))
instructions = [CompiledInstruction(accounts=[1,2,3], data=b58encode(bytes([9] * 5)), program_id_index=4)]
args = MessageArgs(message_header, accounts_keys, recent_blockhash, instructions)
msg = Message(args)
msg_bytes = msg.serialize()
msg_bytes

In [ ]:
decoded_msg = msg.deserialize(msg_bytes)
assert decoded_msg.header == message_header
assert decoded_msg.recent_blockhash == recent_blockhash
print_message(decoded_msg)

In [ ]:
signatures = [
    b58encode(bytes([1] * SIG_LENGTH)),
    b58encode(bytes([2] * SIG_LENGTH)),
];
# Note: Signatures will be added to transaction on each run
tx = Transaction.populate(msg, signatures)
tx_msg = tx.compile_message()
assert tx.serialize_message() == tx_msg.serialize()
assert len(tx_msg.instructions) == len(tx.instructions) == 1
assert tx_msg.recent_blockhash == tx.recent_blockhash == recent_blockhash
print_message(tx_msg)